In [52]:
import pandas as pd
import numpy
df = pd.read_csv('short.csv')


In [23]:
array = numpy.asarray(df)

In [25]:
df.original[1]

'person horse jump broken airplane .'

In [34]:
def calculate_wordsimilarity(source_text,suspicious_text):
    import nltk
    from nltk.tokenize import word_tokenize
    tokenized_suspicious_text = word_tokenize(suspicious_text.lower())
    tokenized_source_text = word_tokenize(source_text.lower())
    #print(tokenized_suspicious_text)
    #print(tokenized_source_text)
    from nltk.corpus import stopwords
    from string import punctuation
    stopwords = stopwords.words('english')
    #print(stopwords)
    punctuation = list(punctuation)
    #print(punctuation)
    cleaned_tokenized_suspicious_text = [token for token in tokenized_suspicious_text if token not in stopwords 

                      and token not in punctuation]

    cleaned_tokenized_source_text = [token for token in tokenized_source_text if token not in stopwords 

                      and token not in punctuation]

    #print(cleaned_tokenized_suspicious_text)
    #print(cleaned_tokenized_source_text)
    words_from_suspicious=nltk.pos_tag(cleaned_tokenized_suspicious_text)
    words_from_source=nltk.pos_tag(cleaned_tokenized_source_text)
    #print(words_from_suspicious)
    #print(words_from_source)
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import wordnet as wn
    lemmatizer = WordNetLemmatizer()

    def nltk_tag_to_wordnet_tag(nltk_tag):
        if nltk_tag.startswith('J'):
            return "a"
        elif nltk_tag.startswith('V'):
            return "v"
        elif nltk_tag.startswith('N'):
            return "n"
        elif nltk_tag.startswith('R'):
            return "r"
        else:         
            return "other"  
    def lemmatizing_with_wntag(words):
        lematized_words = list()
        for word, pos in words:
            temp = tuple()
            wn_tag = nltk_tag_to_wordnet_tag(pos)
            if wn_tag != "other":
                lemmatized_word = lemmatizer.lemmatize(word,wn_tag)
            else:
                lemmatized_word = lemmatizer.lemmatize(word)
            temp = (lemmatized_word,wn_tag)
            lematized_words.append(temp)
        return lematized_words
    words_suspicious_with_wntag = lemmatizing_with_wntag(words_from_suspicious)
    words_source_with_wntag = lemmatizing_with_wntag(words_from_source)
    #print(words_suspicious_with_wntag)
    #print(words_source_with_wntag)
    #len(words_suspicious_with_wntag)
    def finding_synsets(token):
        synsets = dict()
        for word , pos in token:
            if pos.startswith('j'):
                tb=wn.synsets(word,wn.ADJ)
            elif pos.startswith('v'):
                tb=wn.synsets(word,wn.VERB)
            elif pos.startswith('n'):
                tb=wn.synsets(word,wn.NOUN)
            elif pos.startswith('r'):
                tb=wn.synsets(word,wn.ADV)
            else:
                tb=wn.synsets(word)

            synsets[word]=tb
        return synsets
    synsets_of_suspicious = finding_synsets(words_suspicious_with_wntag)
    #print(synsets_of_suspicious)
    def finding_similar_word(synsets):
        similar_word = dict()
        for word in synsets:
            synset_word = synsets[word]
            temp = list()

            for syn in synset_word:
                for l in syn.lemmas():
                    temp.append(l.name().lower())

            temp = list(set(temp))
            similar_word[word]=temp
        return similar_word
    similar_words_suspicious = finding_similar_word(synsets_of_suspicious)
    #print(similar_words_suspicious)
    #len(similar_words_suspicious)
    def checking_similar_with_source(token_from_source):
        modified_source_token =list()
        for word , pos in words_source_with_wntag:
            temp =tuple()
            #print(word,pos)
            for key in similar_words_suspicious:
                #print(key)
                if word in similar_words_suspicious[key]:
                    temp=(key,pos)
                    #break
            #print(len(temp))
            if len(temp) != 0 :
                modified_source_token.append(temp)
            else:
                temp = (word,pos)
                modified_source_token.append(temp)
            #print(temp)
        return modified_source_token
    new_words_source_with_wntag = checking_similar_with_source(words_source_with_wntag)
    #print(new_words_source_with_wntag)
    #len(new_words_source_with_wntag)
    def jaccard_similarity(list1, list2):
        s1 = set(list1)
        s2 = set(list2)
        return float(len(s1.intersection(s2)) / len(s1.union(s2)))
    list1 = words_suspicious_with_wntag
    list2 = new_words_source_with_wntag
    similarity=jaccard_similarity(list1, list2)
    return similarity
calculate_wordsimilarity("man blond-hair , brown shirt drinking public water fountain ","blond man drinking water fountain.")

0.4444444444444444

In [50]:
word_similarity = list()
for i in range(0,len(df)):
    original = df.original[i]
    suspicious = df.suspicious[i]
    #function which calculate similarity feature
    value = calculate_wordsimilarity(original,suspicious)
    word_similarity.append(value)
df['word_similarity'] = word_similarity
df.to_csv('shortdata_with_wordsimilarity.csv',index=False)

In [49]:
df

,original,suspicious,class,word_similarity
0,person horse jump broken airplane .,"person diner , ordering omelette .",0,0.125000
1,person horse jump broken airplane .,"person outdoors , horse .",1,0.333333
2,Children smiling waving camera,child present,1,0.200000
3,Children smiling waving camera,kid frowning,0,0.200000
4,boy jumping skateboard middle red bridge .,boy skate sidewalk .,0,0.125000
...,...,...,...,...
494,People orange vest blue pant yellow stripe bot...,runner waited start race,1,0.071429
495,young man young lady dancing carpeted floor pi...,people dance sidewalk .,0,0.000000
496,young man young lady dancing carpeted floor pi...,two people dancing .,1,0.066667
497,skier wearing yellow jumpsuit sliding across y...,skier wearing jumpsuit .,1,0.428571


In [46]:
df

,original,suspicious,class,word_similarity
0,person horse jump broken airplane .,"person diner , ordering omelette .",0,0.125000
1,person horse jump broken airplane .,"person outdoors , horse .",1,0.333333
2,Children smiling waving camera,child present,1,0.200000
3,Children smiling waving camera,kid frowning,0,0.200000
4,boy jumping skateboard middle red bridge .,boy skate sidewalk .,0,0.125000
...,...,...,...,...
494,People orange vest blue pant yellow stripe bot...,runner waited start race,1,0.071429
495,young man young lady dancing carpeted floor pi...,people dance sidewalk .,0,0.000000
496,young man young lady dancing carpeted floor pi...,two people dancing .,1,0.066667
497,skier wearing yellow jumpsuit sliding across y...,skier wearing jumpsuit .,1,0.428571
